In [1]:
import glob, json, os, re
import random
import numpy as np
from scipy import stats, signal
from datetime import datetime
from PIL import Image
import pytesseract
import pymysql
import sqlalchemy as SQL
from urllib.parse import quote_plus as QP

from IPython.display import clear_output, display
import ipywidgets as widgets
from ipyevents import Event

import matplotlib.pyplot as plt
import seaborn

%matplotlib inline
seaborn.set_theme()

HOME = os.path.expanduser('~')

In [2]:
def get_sql_url():
    sql_secrets_path = os.path.join(HOME, 'Documents', 'sql_secrets.json')
    with open(sql_secrets_path,'r') as SECRETS:
        SECRETS = json.load(SECRETS)
        SQL_PASSWD = SECRETS['PASSWD']
        SQL_USER = SECRETS['USER']
        SQL_HOST = SECRETS['HOST']
        SQL_PORT = SECRETS['PORT']
        SQL_DB = SECRETS['DB']

    return f"mysql+pymysql://{SQL_USER}:{QP(SQL_PASSWD)}@{SQL_HOST}:{SQL_PORT}/{SQL_DB}"

SQL_URL = get_sql_url()
SQL_ENGINE = SQL.create_engine(SQL_URL)

%load_ext sql
%sql $SQL_URL

In [3]:
class History():
    '''
    A class for iterating through database rows, maintaining
    "back" and "forward" stacks.
    '''
    def __init__(self, cursor):
        self.cursor = cursor
        self.back = []
        self.forward = []
        self.setCurrent(self.getNext())

    def getNext(self):
        return self.cursor.fetchone()

    def goForward(self):
        if len(self.forward) > 0:
            self.back.append(self.current)
            self.setCurrent(self.forward.pop())
        else:
            nextRecord = self.getNext()
            if nextRecord is not None:
                self.back.append(self.current)
                self.setCurrent(nextRecord)

    def goBack(self):
        if len(self.back) > 0:
            self.forward.append(self.current)
            self.setCurrent(self.back.pop())

    def setCurrent(self, current):
        self.current = current
        self.update_needed = True
            
    def getCurrent(self):
        self.update_needed = False
        return self.current

    def asHTML(self):
        BACK = '<br>'.join([str(S[1]) for S in self.back])
        FWD = '<br>'.join([str(S[1]) for S in self.forward])
        CUR = self.current[1]
        return (f"<table width=100%><tr><td>current: {CUR}</td><td></td></tr>" \
                + f"<tr><td width=50%>Back:<br>{BACK}</td>" \
                + f"<td width=50%>Forward:<br>{FWD}</td></tr></table>")

class TagIndicator(widgets.HTML):
    def __init__(self, fields, sql_cursor=None):
        super().__init__("")
        self.tags = [-1]*len(fields)
        self.fields = fields
        self.update_html()
        self.cursor = sql_cursor
    
    def get_tags(self):
        return self.tags
        
    def set_all_tags(self, tags):
        self.tags = tags
        self.update_html()
        
    def set_unset_tags(self, new):
        self.tags = [old if (old is not None and old > -1) else new for old in self.tags]
        self.update_html()
    
    def set_tag(self, idx, value):
        self.tags[idx] = value
        self.update_html()
    
    def update_html(self):
        c = lambda tag: ('gray', 'green', 'red')[tag+1]
        cols = [f"<td style='background-color:{c(tag)}'>{self.fields[i]} {tag}</td>" for i, tag in enumerate(self.tags)]
        #cols = [f"<td>{self.fields[i]} {tag}</td>" for i, tag in enumerate(self.tags)]
        self.value = f"<table><tr>{''.join(cols)}</tr></table>"

In [5]:
with SQL_ENGINE.connect().execution_options(autocommit=True) as conn:
    sql_cmd = "SELECT `ImageID`, `Path`, `DateTime` FROM `Metadata` ORDER BY `DateTime`;"
    cursor = conn.execute(SQL.text(sql_cmd))

    
    INFO = TagIndicator(["MotionBlur", "Obscured"])
    POS = History(cursor)
    IMAGE = widgets.Image(format='png', width=800)

    def update_image():
        if POS.update_needed:
            current = POS.getCurrent()
            path = f"/home/jmp/Data/Storage/AlertWF/Brightwood/{current[1]}"
            INFO.set_all_tags([-1, -1])
            IMAGE.set_value_from_file(path)

    def handle_event(evt):
        if evt['code'] in ['ArrowUp', 'ArrowLeft']:
            POS.goBack()
        elif evt['code'] in ['ArrowDown', 'ArrowRight']:
            POS.goForward()
        elif evt['key'] == 'Enter':
            ## Picks up 'NumpadEnter' as well
            POS.goForward()
        elif evt['code'].startswith('Numpad'):
            idx = int(evt['key'])-1
            val = 0 if evt['ctrlKey'] else 1
            if idx >= 0:
                INFO.set_tag(idx%3, val)
            if idx < 3:
                POS.goForward()
        update_image()

    d = Event(source=IMAGE, watched_events=['keydown'])
    d.on_dom_event(handle_event)
    update_image()
    display(IMAGE, INFO)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x06\xdd\x00\x00\x03\xdc\x08\x06\x00\x00\x00-\xc5ta\x…

TagIndicator(value="<table><tr><td style='background-color:gray'>MotionBlur -1</td><td style='background-color…